In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 66.3 MB/s eta 0:00:00


In [4]:
!pip install -q selenium webdriver-manager beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 13.5 MB/s eta 0:00:00


In [5]:
# Célula de Instalação do Chrome (execute uma vez no início)

# 1. Atualiza a lista de pacotes
!apt-get update

# 2. Instala as dependências necessárias para o Chrome rodar
!apt-get install -y libu2f-udev fonts-liberation

# 3. Baixa o pacote de instalação oficial do Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

# 4. Instala o pacote
!dpkg -i google-chrome-stable_current_amd64.deb

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,084 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,812 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy In

In [7]:
PLAYLIST_URLS = [
    "https://youtube.com/playlist?list=PLwASK72qOY0ilVdf85SqD3KQlOO5k2DEV&si=pHUpG4x2Lji5wwl4",
    "https://youtube.com/playlist?list=PLwASK72qOY0hj67HwBbZhEce3vSC63bJo&si=kxdwoECGxt0_819m",
    "https://youtube.com/playlist?list=PLwASK72qOY0gKC-CuZw-BqupeoW9f6JnA&si=BqFhtYd7475Ng8WZ",
    "https://youtube.com/playlist?list=PLwASK72qOY0jzcQHvT9NcNh3LAHHXgic3&si=9OH4HKbBJicOrY9l"
]

In [11]:
import pandas as pd
import os
import yt_dlp  # Esta é a biblioteca-chave

# --- CONFIGURAÇÃO ---
DRIVE_BASE_FOLDER = '/content/drive/MyDrive/musica_ext/'
# A pasta OUTPUT_FOLDER não era usada, então mantive assim.
VIDEOS_LIST_CSV = os.path.join(DRIVE_BASE_FOLDER, 'lista_completa_videos.csv')

PLAYLIST_URLS = [
    "https://youtube.com/playlist?list=PLwASK72qOY0ilVdf85SqD3KQlOO5k2DEV&si=pHUpG4x2Lji5wwl4",
    "https://youtube.com/playlist?list=PLwASK72qOY0hj67HwBbZhEce3vSC63bJo&si=kxdwoECGxt0_819m",
    "https://youtube.com/playlist?list=PLwASK72qOY0gKC-CuZw-BqupeoW9f6JnA&si=BqFhtYd7475Ng8WZ",
    "https://youtube.com/playlist?list=PLwASK72qOY0jzcQHvT9NcNh3LAHHXgic3&si=9OH4HKbBJicOrY9l"
]

def get_playlist_videos_with_yt_dlp(playlist_url):
    """
    Usa yt-dlp para extrair os metadados da playlist de forma eficiente.
    Este método vai buscar TODOS os vídeos (os 75, no seu caso).
    """
    print(f"\nBuscando vídeos na playlist: {playlist_url}")
    videos = []

    # Opções para o yt-dlp:
    # 'extract_flat': Obtém a lista da playlist sem carregar cada vídeo (MUITO rápido).
    # 'skip_download': Não baixa o vídeo.
    # 'quiet': Não polui o console.
    ydl_opts = {
        'extract_flat': True,
        'skip_download': True,
        'quiet': True,
        'force_generic_extractor': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Extrai a informação da playlist
            result = ydl.extract_info(playlist_url, download=False)

            # 'entries' é a lista de todos os vídeos na playlist
            if 'entries' in result:
                for entry in result['entries']:
                    if entry:
                        videos.append({
                            'url': entry.get('url'), # URL do vídeo
                            'title': entry.get('title'), # Título do vídeo
                            'playlist_url': playlist_url
                        })

        print(f"-> Encontrados {len(videos)} vídeos.") # Aqui deve mostrar 75!

    except Exception as e:
        print(f"-> ERRO ao processar {playlist_url}: {e}")

    return videos


def main():
    print(f"Iniciando busca das playlists...\n")
    all_videos = []

    for url in PLAYLIST_URLS:
        videos = get_playlist_videos_with_yt_dlp(url)
        all_videos.extend(videos)

    if not all_videos:
        print("⚠ Nenhum vídeo encontrado.")
        return

    df = pd.DataFrame(all_videos)

    # Garante que a pasta de destino exista
    output_dir = os.path.dirname(VIDEOS_LIST_CSV)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    df.to_csv(VIDEOS_LIST_CSV, index=False, encoding='utf-8')
    print(f"\n✅ Lista de {len(df)} vídeos salva com sucesso em:\n{VIDEOS_LIST_CSV}")


if __name__ == "__main__":
    main()

Iniciando busca das playlists...


Buscando vídeos na playlist: https://youtube.com/playlist?list=PLwASK72qOY0ilVdf85SqD3KQlOO5k2DEV&si=pHUpG4x2Lji5wwl4
-> Encontrados 75 vídeos.

Buscando vídeos na playlist: https://youtube.com/playlist?list=PLwASK72qOY0hj67HwBbZhEce3vSC63bJo&si=kxdwoECGxt0_819m
-> Encontrados 75 vídeos.

Buscando vídeos na playlist: https://youtube.com/playlist?list=PLwASK72qOY0gKC-CuZw-BqupeoW9f6JnA&si=BqFhtYd7475Ng8WZ
-> Encontrados 75 vídeos.

Buscando vídeos na playlist: https://youtube.com/playlist?list=PLwASK72qOY0jzcQHvT9NcNh3LAHHXgic3&si=9OH4HKbBJicOrY9l
-> Encontrados 74 vídeos.

✅ Lista de 299 vídeos salva com sucesso em:
/content/drive/MyDrive/musica_ext/lista_completa_videos.csv
